In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from matplotlib import pyplot as plt
import pandas as pd
from wordcloud import WordCloud
from uuid import uuid4


# Extends pandas txt accessor
from utils import TextAccessor

In [15]:
csv = Path().cwd().parent/"Clean/all.csv"
df_princiapl = pd.read_csv(str(csv)).head()

In [ ]:
df_princiapl